In [2]:
import numpy as np
#from utils2 import *
from tensorflow.keras.layers import *
from tensorflow.keras import *
import tensorflow as tf

import matplotlib.pyplot as plot

# https://numpy.org/doc/stable/user/numpy-for-matlab-users.html
import numpy.matlib as M

from keras import backend as K


# Load MATLAB variables saved to the file data.mat. (Note: When saving arrays to data.mat in MATLAB/Octave,
# use a recent binary format. scipy.io.loadmat will create a dictionary with the saved arrays and further information.)
import scipy.io as io


Input=io.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Our Papers\Matlabcodes__Dynamic Structure\TrainingData.mat")
H_input=Input['TrainingData']
print(H_input.shape)

Input=io.loadmat(r"C:\Users\Jamal\OneDrive\Jamal\MATLAB Code\Our Papers\Matlabcodes__Dynamic Structure\TrainingUserLabels.mat")
Label=Input['TrainingUserLabels']
print(Label.shape)
num_classes=Label.shape[1]

(200000, 144, 5, 3)
(200000, 720)


In [3]:
# imperfect CSI is used to output the vrf
imperfect_CSI = tf.keras.Input(name='imperfect_CSI', shape=(H_input.shape[1:4]), dtype=tf.float32)
temp = BatchNormalization()(imperfect_CSI)
temp = Flatten()(temp)
#temp = BatchNormalization()(temp)
temp = Dense(4*num_classes, activation='relu')(temp)
#temp = BatchNormalization()(temp)
temp = Dense(4*num_classes, activation='relu')(temp)
#temp = BatchNormalization()(temp)
output= Dense(num_classes, name='FC_layer4')(temp)
output=Activation('sigmoid')(output)
model = Model(inputs=[imperfect_CSI], outputs=output)

def cat_loss(y_true, y_pred):
    loss = K.mean(K.binary_crossentropy(y_true[:, 0:num_classes - 1], y_pred[:, 0:num_classes - 1]), axis=-1) \
             + K.mean(K.binary_crossentropy(y_true[:, num_classes:2 * num_classes - 1], y_pred[:, num_classes:2 * num_classes - 1]), axis=-1)
    return loss

adam = tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
model.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy())
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 imperfect_CSI (InputLayer)  [(None, 144, 5, 3)]       0         
                                                                 
 batch_normalization (BatchN  (None, 144, 5, 3)        12        
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 2160)              0         
                                                                 
 dense (Dense)               (None, 2880)              6223680   
                                                                 
 dense_1 (Dense)             (None, 2880)              8297280   
                                                                 
 FC_layer4 (Dense)           (None, 720)               2074320   
                                                             

In [4]:
Label = tf.cast(Label, tf.float32)
model.fit(H_input, Label,batch_size=500, epochs=50)

Epoch 1/50
400/400 [==============================] - 159s 398ms/step - loss: 0.3948
Epoch 2/50
400/400 [==============================] - 176s 439ms/step - loss: 0.3031
Epoch 3/50
400/400 [==============================] - 182s 455ms/step - loss: 0.2623
Epoch 4/50
400/400 [==============================] - 204s 510ms/step - loss: 0.2339
Epoch 5/50
400/400 [==============================] - 288s 720ms/step - loss: 0.2127
Epoch 6/50
400/400 [==============================] - 289s 723ms/step - loss: 0.1960
Epoch 7/50
400/400 [==============================] - 233s 582ms/step - loss: 0.1827
Epoch 8/50
400/400 [==============================] - 197s 493ms/step - loss: 0.1717
Epoch 9/50
400/400 [==============================] - 193s 483ms/step - loss: 0.1623
Epoch 10/50
400/400 [==============================] - 187s 468ms/step - loss: 0.1544
Epoch 11/50
400/400 [==============================] - 179s 447ms/step - loss: 0.1474
Epoch 12/50
400/400 [==============================] - 178s 445

In [5]:
model.save("my_h5_model12.h5")


In [6]:
print(H_input[1].shape)
tf.math.round(model.predict(H_input[0:1]))-Label[0:1]
#print(Label[0:1])

(144, 5, 3)


<tf.Tensor: shape=(1, 720), dtype=float32, numpy=
array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
 